### This file is coded in order to use the anomaly detection autoencoder on data

In [13]:
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import pickle

### Load the model

In [14]:
autoencoder = load_model('anomaly_detection/output/autoencoder.model')
images = pickle.loads(open('/media/juan/juan1/pyimage_univ/Autoencoders/anomaly_detection/output/images.pickle', "rb").read())

### Make predictions (get decoded images)

In [15]:
decoded = autoencoder.predict(images)
#Save the reconstructions errors to a list
errors = []

213/213 [==============================] - 1s 2ms/step


In [16]:
#Go through the images and their reconstructions (decoded)
for (image, recon) in zip(images, decoded):
    #Compute the mean squared error (MSE) and add them to the errors list
    mse = np.mean((image-recon)**2)
    errors.append(mse)

In [17]:
errors[:5]

[0.0011744336, 0.0005435257, 0.0012951262, 0.0005053923, 0.0008721597]

### Anomaly detection

##### To separate the normal images from the anomalies is necessary to define a threshold. Any data point that has a bigger error than our threshold will be classsified as an anomaly. For the threshold calculation, we will use quantiles

In [18]:
thresh = np.quantile(errors, 0.999) #1% of the data will be classified as anomaly

In [19]:
idxs = np.where(np.array(errors)>= thresh)

In [23]:
#Initialize the outputs array
outputs = None

#loop ober the indexes of images with high MSE 
for i in idxs:
    #get the original image and the reconstructed image
    original = (images[i]*255).astype("uint8") #In order to make the images work with cv2
    recon = (decoded[i]*255).astype("uint8")

    #Stack the original and reconstructed
    output = np.hstack([original, recon])

    #Add all the anomaly images together
    if outputs is None:
        outputs = output #the hstack
    
    else:
        outputs = np.vstack([outputs, output])#add the images to the None space

cv2.imshow('Outputs', outputs)
cv2.waitKey(0)
cv2.destroyAllWindows()